<a href="https://colab.research.google.com/github/HighestPotential/HuggingFaceCourseAgents/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Key Functionality**

**Web Search :**
Leverages DuckDuckGoSearchTool to dynamically fetch real-time search results for user queries.

**Result Grouping :**
Aggregates raw search results into large, manageable chunks (e.g., 1,000 characters per group) to create structured Document objects for downstream processing.


**Text Splitting :**
Splits grouped documents into smaller, overlapping segments (e.g., 200 tokens with 10-token overlap) to optimize context retention and retrieval efficiency.



**BM25 Retrieval :**
Uses the BM25 algorithm to rank and retrieve the top 5 most relevant text chunks based on query similarity, ensuring precise information filtering.



**Output :**
Returns results as clean, formatted snippets (e.g., "Relevant Content 1: ...") for easy interpretation and integration into workflows.



In [1]:
pip install smolagents -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install langchain-community rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [10]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever
from smolagents import CodeAgent, HfApiModel, VisitWebpageTool, DuckDuckGoSearchTool

In [37]:
class LawExtractionTool(Tool):
    name = "LawExtractionSearchTool"
    description = "Performs web search and uses BM25 to retrieve relevant results for a query, use 'def forward' for that"
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform.",
        }
    }
    output_type = "string"


    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"
        search_results = DuckDuckGoSearchTool().forward(query)

        grouped_results = [
        " ".join(search_results[i:i+1000])
        for i in range(0, len(search_results), 1000)
        ]

        docs = [Document(page_content=result) for result in grouped_results]


        self.retriever = BM25Retriever.from_documents(
            docs, k=3
        )

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=200,
            chunk_overlap=10,
            separators=["\n\n"],
        )
        split_docs = text_splitter.split_documents(docs)

        retriever = BM25Retriever.from_documents(split_docs, k=5)
        retrieved_docs = retriever.invoke(query)

        return "\n\n".join(
            [f"Relevant Content {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)]
        )


In [38]:
agent = CodeAgent(tools=[LawExtractionTool()], model=HfApiModel(), max_steps=5)
response = agent.run(
    "How long are you allowed to be in the USA without a visia if you are a Canadian citizen?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How long are you allowed to be in the USA without a visia if you are a Canadian citizen?                        │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = LawExtractionSearchTool(query="visa-free entry duration for Canadian citizens in the USA")              
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Relevant Content 1:
t r a n s f e r s   w i t h i n   t h e   s a m e   o r g a n i z a t i o n .   E - 2   V i s a :   F o r   i n v e
s t o r s   s t a r t i n g   o r   m a n a g i n g   a   b u s i n e s s . 
 
 [ U S A   V i s a   f o r   C a n a d i a n   c i t i z e n s   2 0 2 5   -   V i s a   L i s t ] ( h t t p s : / 
/ v i s a l i s t . i o / u n i t e d - s t a t e s - o f - a m e r i c a / v i s a - r e q u i r e m e n t s / c a
n a d a ) 
 U S A   V i s a   f r e e   a p p l i c a t i o n   p r o c e s s   f o r   C a n a d i a n   c i t i z e n s   i 
s   q u i c k   a n d   e a s y ,   a s   l o n g   a s   t h e   n e c e s s a r y   d o c u m e n t s   a n d   i
n f o r m a t i o n   a r e   p r o v i d e d   a c c u r a t e l y .   B y   f o l l o w i n g   t h e s e   4   s
t e p s ,   C a n a d a   c i t i z e n s   c a n   e n s u r e   a   s m o o t h   a n d   h a s s l e - f r e e  
p r o c e s s   i n   o b t a i n i n g   a n   U S A   t o u r i s t   v i s a .

Relevant Content 2:
  a   v i s a .   V i s i t o r s   t o   C a n a d a   w h o   w i s h   t o   t r a v e l   o n   t o   t h e   U
n i t e d   S t a t e s ,   a n d   w h o   a r e   c i t i z e n s   o f   c o u n t r i e s   i n c l u d e d   i
n   t h e   p r o g r a m   a r e   e l i g i b l e   f o r   v i s a - f r e e   e n t r y   t o   t h e   U n i t
e d   S t a t e s . 
 
 [ C a n a d i a n s   v i s i t i n g   t h e   U S A   V i s a   I n f o r m a t i o n   -   C a n a d a   t o   
U S A ] ( h t t p s : / / w w w . c a n a d a t o u s a . c o m / i m m i g r a t i o n / v i s i t i n g - t h e -
u s a / ) 
 A s   a   C a n a d i a n   c i t i z e n ,   y o u   d o   n o t   r e q u i r e   a   v i s a   t o   e n t e r 
t h e   U n i t e d   S t a t e s   t e m p o r a r i l y   f o r   b u s i n e s s   o r   f o r   p l e a s u r e
,   b u t   a   p a s s p o r t   i s   a l w a y s   r e q u i r e d   n o   m a t t e r   h o w   y o u   e n t e
r ,   b y   a i r   o r   g r o u n d .   C a n a d i a n s   c o m i n g   t o   t h e   U S   f o r   v a c a t i
o n   a r e   g e n e r a l l y   a l l o w e d   t o   s t a y   u p   t o   a   m a x i m u m   6   m o n t h s  
o r   1 8 2   d a y s . 
 
 [ C a n   C a n a d i a n s   M o v e   t o   t h e   U . S .   W i t h o u t   a   V i s a ? ] ( h t t p s : / / 
r j i m m i g r a t i o n l a w . c o m / r e s o u r c e s / c a n - c a n a d i a n s - m o v e - t o - t h e - u
- s - w i t h o u t - a - v i s a / ) 
 L i v i n g   w i t h   a   U . S .   c i t i z e n   s p o u s e ;   S t a n d a r d   U . S .   V i s a   O p t 
i o n s   f o r   C a n a d i a n s :   T N   V i s a   ( u n d e r   N A F T A / U S M C A ) :   F o r   c e r t a
i n   p r o f e s s i o n a l s   l i k e   e n g i n e e r s ,   a c c o u n t a n t s ,   a n d   s c i e n t i s
t s .   H - 1 B   V i s a :   F o r   s p e c i a l i z e d   j o b s   t h a t   r e q u i r e   a   d e g r e e .
L - 1   V i s a :   F o r   c o m p a n y  

Relevant Content 3:
v i s a ,   w h e n   t h e y   m e e t   c e r t a i n   r e q u i r e m e n t s ,   u n d e r   U . S .   l a w s
.   F o r   i n f o r m a t i o n   a b o u t   t r a v e l   w i t h o u t   a   v i s a ,   c l i c k   o n   t h
e   l i n k s   b e l o w .   V i s a   W a i v e r   P r o g r a m   ( V W P )   M e x i c a n   a n d   C a n a d
i a n   N A F T A   P r o f e s s i o n a l   W o r k e r ;   C i t i z e n s   o f   C a n a d a   a n d   B e r m
u d a   .   M o r e   . . . 
 
 [ C a n a d i a n ' s   G u i d e   t o   U S A :   V i s a - F r e e   T r a v e l   -   r j i m m i g r a t i o 
n l a w . c o m ] ( h t t p s : / / r j i m m i g r a t i o n l a w . c o m / r e s o u r c e s / h o w - l o n g -
c a n - i - s t a y - i n - t h e - u s - a s - a - c a n a d i a n - v i s i t o r / ) 
 V i s a - f r e e   t r a v e l   t o   t h e   U . S .   f o r   C a n a d i a n s   c a n   b e   b o t h   c o 
n v e 

[Step 0: Duration 1.01 seconds| Input tokens: 2,093 | Output tokens: 69]

...

[Step 1: Duration 25.60 seconds| Input tokens: 8,897 | Output tokens: 153]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("182 days")                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 182 days

[Step 2: Duration 14.37 seconds| Input tokens: 20,621 | Output tokens: 244]